
# An Example Python Client for the demo-jsonapi API

  - [The {json:api} API convention](#{json:api})
  - [The demo resource server: Courses](#The-Demo-resoure-server:-Courses)
  - [Building the client](#Building-the-client)
    - [OAuth 2.0](#OAuth-2.0)
    
## The {json:api} API convention

[{json:api}](http://jsonapi.org/) establishes a shared convention for RESTful API requests/responses that may
help get closer to [HATEOAS](https://en.wikipedia.org/wiki/HATEOAS) constraints. Basically this means any client that connects to the API
should be able to use it without any external knowledge.

## The Demo resoure server: Courses

**DEPRECATION NOTICE:** We've dropped Mulesoft and are instead using Django Rest Framework with JSON API (DJA). The demo app is [here](https://gitlab.cc.columbia.edu/ac45/django-training) (accessible only to internal staff, sorry). The Widgets and Locations demo is no longer available.


## Building the client with jsonapi_requests

There are a [couple of {json:api} Python libraries](http://jsonapi.org/implementations/#client-libraries-python) so let's
try the first one, [jsonapi-requests](https://github.com/socialwifi/jsonapi-requests/)

```sh
pip3 install jsonapi-requests
```

### Using the ORM

It looks like jsoanpi-requests ORM is immature and may not properly support adding/updating
relationships....

## OAuth 2.0 

Before you can do anything with the Resource Server, you need to use OAuth 2.0 to get a _bearer_ token which 
we'll send in an `Authorization:` header.

When using [OAuth 2.0](https://tools.ietf.org/html/rfc6749) (no matter what the flow), the client app (this notebook) gets an *access token* (and optionally some other tokens like *refresh\_token* and even an OpenID Connect *id\_token* which identifies the end user to the client app -- when using the *openid* scope). 

Once the client app has the access token, it then adds it to the HTTP request that it sends to the Resource Server in one of two ways, either
1. in an HTTP header: `Authorization: Bearer <token>` (preferred) or,
2. as a query parameter: `https://example.com/resource?access_token=<token>`.
But not both!

~~Let's try to use [requests-oauthlib](http://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html) for this.~~

```sh
#pip3 install requests_oauthlib
```

NOPE: requests-oauthlib is not as good as oauth2_client. It doesn't seem to open the redirect_uri callback listener.

```sh
pip3 install oauth2-client
```

In [ ]:
from oauth2_client.credentials_manager import CredentialManager, ServiceInformation, OAuthError
from jwcrypto import jwt, jwk
import webbrowser
import requests
import jsonapi_requests
import json
import base64
from pprint import pprint, pformat
import time
import logging

# turn on requests logging so we can what jsonapi_requests is doing
debug = 1
import http.client as http_client
http_client.HTTPConnection.debuglevel = debug
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG if debug else logging.INFO)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG if debug else logging.INFO)
requests_log.propagate = True

# subclass CredentialManager to get the id_token which is used later.
class OpenIdCredentialManager(CredentialManager):
    def __init__(self, service_information, proxies=None):
        super(OpenIdCredentialManager, self).__init__(service_information, proxies)
        self.id_token = None

    def _process_token_response(self,  token_response, refresh_token_mandatory):
        id_token = token_response.get('id_token')
        super(OpenIdCredentialManager, self)._process_token_response(token_response, refresh_token_mandatory)
        self.id_token = id_token

oauth2_server = 'https://oauth-test.cc.columbia.edu'

# demo_client lacks 'demo-netphone-admin' scope which is required to create new courses.
# demo_trusted_client allows this scope, but only if the user has the scope from Shibboleth.
# creds = {'id':'demo_client','secret':'b322573a7176A49FCBEF46554d3381d5'}
creds = {'id': 'demo_trusted_client', 'secret': 's9ht0XNvHEkvXfUhVD1Ka9DtXFxRHfTm'}
# a meaningless redirect URI that the OAuth2 client starts a server on to catch the response.
# it must be registered with the above client in OAuth2 server.
redirect_uri = 'http://localhost:5432/oauth2client'

service_information = ServiceInformation(
    authorize_service=oauth2_server + '/as/authorization.oauth2',
    token_service=oauth2_server + '/as/token.oauth2',
    client_id=creds['id'],
    client_secret=creds['secret'],
    scopes=['auth-columbia', 'create','read','update','delete','demo-netphone-admin', 'openid', 'profile'],
    # skip_ssl_verifications=False,
)

manager = OpenIdCredentialManager(service_information) # initialize the OAuth 2.0 client
authUrl = manager.init_authorize_code_process(redirect_uri, 'state_test')
webbrowser.open_new(authUrl)
code = manager.wait_and_terminate_authorize_code_process()
manager.init_with_authorize_code(redirect_uri, code)
print('access token: %s'%manager._access_token)

# import the list of public keys from the OAuth server
keysetText = requests.get(oauth2_server + '/pf/JWKS').text
keyset=jwk.JWKSet().import_keyset(keysetText)
# validate the OIDC id_token
id_token = manager.id_token
# check that openid token is valid, first just unencoding and printing it, then validating it.
print("id_token: %s"%id_token)
if id_token:
    splits = id_token.split('.')
    for i in range(3):
        missing_padding = 4 - len(splits[i]) % 4 # b64 encoding needs to be padded
        if missing_padding: 
            splits[i] += '='* missing_padding
    hdr = json.loads(base64.b64decode(splits[0]))
    body = json.loads(base64.b64decode(splits[1]))
    print('Header:\n%s'%pformat(hdr))
    print('Body:\n%s'%pformat(body))
    # the signature is binary junk
    if 'exp' in body and int(time.time()) > body['exp']:
        print("The id_token is expired. The following jwcrypto validation will fail.")
    try:
        et = jwt.JWT(key=keyset, jwt=id_token)
        st = jwt.JWT(key=keyset, jwt=et.serialize())
        print('This is a %s id_token'%('valid' if st.token.is_valid else 'invalid'))
        print('header:\n%s\nbody:\n%s'%(pformat(json.loads(st.header)),pformat(json.loads(st.claims))))
    except Exception as e:
        print('Exception %s: %s'%(type(e),e))

In [ ]:
# set up my custom auth module which provides the access token from oauth2_client.
class MyAuth(requests.auth.AuthBase):
    def __call__(self, r):
        r.headers['Authorization'] = 'Bearer ' + manager._access_token
        return r

# login to the demo API which is running locally
api = jsonapi_requests.orm.OrmApi.config({
    'API_ROOT': 'http://localhost:8000/v1',
    'AUTH': MyAuth(),
    'VALIDATE_SSL': False,
    'TIMEOUT': 1,
})

# define some classes that correspond to the resource server types
class Course(jsonapi_requests.orm.ApiModel):
    class Meta:
        type = 'courses'
        api = api
    course_name = jsonapi_requests.orm.AttributeField('course_name')
    course_number = jsonapi_requests.orm.AttributeField('course_number')
    course_identifier = jsonapi_requests.orm.AttributeField('course_identifier')
    course_description = jsonapi_requests.orm.AttributeField('course_description')
    last_mod_user_name = jsonapi_requests.orm.AttributeField('last_mod_user_name')
    school_bulletin_prefix_code = jsonapi_requests.orm.AttributeField('school_bulletin_prefix_code')
    subject_area_code = jsonapi_requests.orm.AttributeField('subject_area_code')
    suffix_two = jsonapi_requests.orm.AttributeField('suffix_two')
    course_terms = jsonapi_requests.orm.RelationField('course_terms')
    
    def __str__(self):
        return "%s: %s: %s"%(self.course_identifier, self.course_number, self.course_name)

class CourseTerm(jsonapi_requests.orm.ApiModel):
    class Meta:
        type = 'course_terms'
        api = api
    term_identifier = jsonapi_requests.orm.AttributeField('term_identifier')
    audit_permitted_code = jsonapi_requests.orm.AttributeField('audit_permitted_code')
    exam_credit_flag = jsonapi_requests.orm.AttributeField('exam_credit_flag')
    course = jsonapi_requests.orm.RelationField('course')
    last_mod_user_name = jsonapi_requests.orm.AttributeField('last_mod_user_name')

In [ ]:
# get the list of courses
courses = Course.get_list()
# see how many -- this API's pagination defaults to 10
len(courses)

In [ ]:
for c in courses:
    print('====================\n%s'%(c))
    if c.course_terms:
        print('this course has %d terms:'%len(c.course_terms))
        print(['term: %s'%(t.term_identifier) for t in c.course_terms])
    else:
        print('no terms')

**The following steps will fail if the logged-in user is not a member of the demo-netphone-admin group**

In [ ]:
# now let's try adding a new course:
newC = Course()
newC.course_name = "Motorcycle Maintenance"
newC.course_identifier = "RLGN1001X"
newC.course_number = '12345'
try:
    newC.save() # this is going to fail because of some missing required fields
    print("new Course created with id: %s"%newC.id)
except Exception as e:
    try:
        j = json.loads(e.content)
    except:
        j = 'Error: '+ e.content
    pprint(j)


In [ ]:
# add in the missing fields and try again
newC.course_description = "A seminar about 'Zen and the Art of Motorcycle Maintentance'"
newC.last_mod_user_name = "foo" # shouldn't this be set by the server?
newC.school_bulletin_prefix_code = "00"
newC.subject_area_code = "ZEN"
newC.suffix_two = "22"
try:
    newC.save() # should work this time
    print("new Course created with id: %s"%newC.id)
except Exception as e:
    try:
        j = json.loads(e.content)
    except:
        j = 'Error: '+ e.content
    pprint(j)

In [ ]:
n = Course.from_id(newC.id)
print("New Course %s: %s"%(n.id, n))

In [ ]:
# add some terms to the new course
new_terms = []
for term in ['20181', '20182', '20183']:
    t = CourseTerm()
    t.term_identifier = term
    t.last_mod_user_name = 'foo'
    try:
        t.save()
        print("Saved term ID %s"%t.id)
        newterms.append(t)
    except Exception as e:
        print(e)
# this may not work as jsonapi-requests==0.4.0 is incomplete w.r.t. updating relationships
# see https://github.com/socialwifi/jsonapi-requests/issues/30
# suggested workaround with jsonapi-requests==0.4.1:
# This is still not working in 0.6.0.
newC.course_terms = newterms
newC.save()

In [ ]:
print(['term: %s'%(t.term_identifier) for t in newC.course_terms])

In [ ]:
 # delete the course so it won't cause duplicate errors when we re-run
for t in newC.course_terms:
    t.delete()
newC.delete()